In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext=SQLContext(sc)

## Considerando los logs de acciones realizadas sobre ítems, mostrar el top 10 de ids de ítems que fueron afectados por mayor cantidad usuarios distintos

cargo el DF de contents y de logs y los paso a tipo parquet para poder levantarlos como rdd

In [4]:
df_contents = pd.read_csv('contents.csv', encoding='Latin-1')
df_contents.to_parquet('contents.parquet')
df_contents = sqlContext.read.parquet('contents.parquet')
rdd_contents = df_contents.rdd.cache()

In [7]:
df_logs = pd.read_csv('logs.csv', encoding='Latin-1')
df_logs.to_parquet('logs.parquet')
df_logs = sqlContext.read.parquet('logs.parquet')
rdd_logs = df_logs.rdd.cache()

Realizo un map sobre el rdd de logs para quedarme con el titulo y el usuario como clave y un 1 como valor. Luego hago un filter para eliminar los None de los usuarios y poder quedarme solo con los registrados, para luego hacer un nuevo map para quedarme con la forma titulo como clave y el 1 como valor.

In [8]:
rdd_logs_map = rdd_logs.map(lambda x: ((x[8],x[2]),1)).filter(lambda x: x[0][1] != None).map(lambda x: (x[0][0],x[1]))

Hago un reduceByKey para que me queden los titulos y la cantidad de veces que aparecen. Luego un take para ver la estructura

In [9]:
rdd_logs_reduce = rdd_logs_map.reduceByKey(lambda x,y: x+y).cache()

In [10]:
rdd_logs_reduce.take(10)

[('Ueda', 1),
 ('CategorÃ\xada:One Piece', 1),
 ('Curvas de bezier', 3),
 ('Archivo:Nazca06.jpg', 3),
 ('Angra', 1),
 ('Shane MacGowan', 1),
 ('TIA-568B', 1),
 ('Archivo:Tucuman1.png', 1),
 ('Usuario:148.202.164.58', 1),
 ('Plantilla:ONG', 1)]

Hago un map con el rdd de contenido para quedarme con la forma titulo y id como clave y un 1 como valor.

In [11]:
rdd_contents_map = rdd_contents.map(lambda x: ((x[0],x[1]),1))

Realizo un reduceByKey por clve y id para que me quede solo una repetición del titulo con su id y no me queden registros duplicados, y luego hago un map para que me quede de la forma titulo como calve y el id como valor.

In [12]:
rdd_contents_reduce = rdd_contents_map.reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],x[0][1])).cache()

Realizo el join entre los rdd de contenido y de logs usando titulos como clave

In [13]:
rdd_join = rdd_contents_reduce.join(rdd_logs_reduce)

Luego del join, realizo un map para que me quede de la forma pedida en el enunciado: id del titulo y las repeticiones

In [14]:
rdd_join_map = rdd_join.map(lambda x: (x[1][0],x[1][1])).reduceByKey(lambda x,y: x+y)


verifico la estructura con un take

In [15]:
rdd_join_map.take(2)

[(4435920, 1), (8730900, 1)]

Ahora si, busco la respuesta final haciendo un takeOrdered de los 10 id con mas repeticiones

In [17]:
rdd_join_map.takeOrdered(10, lambda x: -x[1])

[(2054827, 982),
 (1292078, 982),
 (241067, 982),
 (1007123, 982),
 (2271189, 521),
 (19013, 343),
 (951915, 324),
 (314958, 305),
 (4074354, 159),
 (208803, 159)]